In [1]:
%matplotlib widget
from context import spew
from scipy.optimize import curve_fit, minimize
import numpy as np
from shapely.geometry import Point
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import ScalarFormatter, FormatStrFormatter
import scipy.ndimage
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import rc
from mpl_toolkits.mplot3d import axes3d
from matplotlib import colors as mcolors
from matplotlib.collections import PolyCollection
import pandas as pd
import scipy.stats as stats

plt.style.use('ggplot')


rc('font', **{'family': 'serif', 'serif': ['Palatino']})
rc('text', usetex=True)



In [2]:
param = "plume_height"
label = "H"
units = "km"
trial = 7
vals = np.linspace(10000, 40000, 21)

vals = [int(val) for val in vals]

def disp_func(col):
    return col / 1000

eruptions = dict()

In [3]:
for v in vals:
    filename = '../data/%s_trial_%d/%s_trial_%d.txt' % (
        param, trial, param, v)
    eruptions[v] = spew.eruption.Eruption(data=filename, vent=Point(532290, 1382690), test=False)
    eruptions[v].df = eruptions[v].df[(eruptions[v].df["MassArea"] != 0)
                                      & (eruptions[v].df["MassArea"] > 0.1)
                                      & (eruptions[v].df["radius"] < 40000)]
phis = eruptions[vals[0]].phi_labels
rows = int(len(phis) / 2)

In [4]:
centroids = []
import re
for name in phis:
    m1 = re.search('[-+]?[0-9]*\.?[0-9]+(?=,)', name)
    m2 = re.search('[-+]?[0-9]*\.?[0-9]+(?=\))', name)
    centroids.append((float(m1.group(0)) + float(m2.group(0)))/2)
centroids = np.array(centroids)

In [5]:
display(eruptions[40000].df)

,Easting,Northing,Elevation,MassArea,"[-4.5,-4.07143)","[-4.07143,-3.64286)","[-3.64286,-3.21429)","[-3.21429,-2.78571)","[-2.78571,-2.35714)","[-2.35714,-1.92857)",...,"[1.07143,1.5)","[1.5,1.92857)","[1.92857,2.35714)","[2.35714,2.78571)","[2.78571,3.21429)","[3.21429,3.64286)","[3.64286,4.07143)","[4.07143,4.5)",geometry,radius
2917,525487,1421887,0,16.5480,2.390240e-19,1.336890e-17,7.843290e-16,4.599970e-14,2.554500e-12,1.377870e-10,...,10.9088,20.8234,24.7809,19.7933,11.54110,5.38324,2.15453,0.777343,POINT (525487 1421887),39782.981512
2918,527487,1421887,0,16.9539,4.556780e-19,2.423060e-17,1.349480e-15,7.507240e-14,3.954450e-12,2.021950e-10,...,11.1411,20.9514,24.6890,19.6056,11.39570,5.30723,2.12272,0.765664,POINT (527487 1421887),39490.171157
2919,529487,1421887,0,17.2288,6.954510e-19,3.577590e-17,1.925680e-15,1.034810e-13,5.265390e-12,2.599570e-10,...,11.2949,21.0333,24.6262,19.4813,11.30000,5.25735,2.10187,0.758011,POINT (529487 1421887),39297.094269
2920,531487,1421887,0,17.3620,8.497920e-19,4.303480e-17,2.279260e-15,1.204880e-13,6.030930e-12,2.928480e-10,...,11.3683,21.0717,24.5957,19.4219,11.25450,5.23366,2.09196,0.754377,POINT (531487 1421887),39205.224371
2921,533487,1421887,0,17.3481,8.314170e-19,4.217680e-17,2.237780e-15,1.185080e-13,5.942500e-12,2.890790e-10,...,11.3605,21.0677,24.5990,19.4282,11.25940,5.23616,2.09301,0.754761,POINT (533487 1421887),39215.272765
2922,535487,1421887,0,17.1877,6.513030e-19,3.367830e-17,1.822430e-15,9.846250e-14,5.037110e-12,2.500460e-10,...,11.2715,21.0212,24.6360,19.5002,11.31450,5.26487,2.10501,0.759162,POINT (535487 1421887),39327.161326
2923,537487,1421887,0,16.8872,4.084870e-19,2.190900e-17,1.231040e-15,6.910180e-14,3.672800e-12,1.895080e-10,...,11.1025,20.9306,24.7048,19.6369,11.41970,5.31975,2.12795,0.767584,POINT (537487 1421887),39540.025518
2924,539487,1421887,0,16.4583,2.050940e-19,1.161020e-17,6.896500e-16,4.095930e-14,2.303390e-12,1.258320e-10,...,10.8552,20.7936,24.8018,19.8365,11.57460,5.40074,2.16186,0.780032,POINT (539487 1421887),39852.247339
3013,519487,1419887,0,17.1653,6.348750e-19,3.289220e-17,1.783440e-15,9.655160e-14,4.949420e-12,2.462010e-10,...,11.2607,21.0150,24.6401,19.5090,11.32140,5.26852,2.10654,0.759725,POINT (519487 1419887),39338.703817
3014,521487,1419887,0,18.0588,2.356850e-18,1.101790e-16,5.374020e-15,2.612950e-13,1.202980e-11,5.367530e-10,...,11.7438,21.2599,24.4333,19.1178,11.02340,5.11370,2.04189,0.736015,POINT (521487 1419887),38733.985310


In [133]:
def gamma_kernel(x, a, b, c):
    return c * (x**a) * np.exp(b * x)


def sse(k, x_data, y_data):
    return sum((1/(x_data**2))*((y_data - (gamma_kernel(x_data, *k))) ** 2))


fit_df = pd.DataFrame(columns=['H', 'Phi', 'a', 'b', 'c', 'data'])

phis[-2] in phis[-2:]


for k, phi in enumerate(phis):
    # fig, axs = plt.subplots(rows, 2, figsize=(
    #     10, 15), facecolor='w', edgecolor='k')
    # axs = axs.ravel()
    for i, (h, erp) in enumerate(eruptions.items()):
        new_df = erp.df
        ma = new_df['MassArea'].values
        wt = new_df[phi].values
        xx = new_df['radius'].values
        yy = ma * (wt / 100)

        pnts = list(set(zip(xx, yy)))
        pnts = sorted(pnts, key=lambda x: x[0])

        x = np.array([p[0] for p in pnts], dtype=np.float64)
        y = np.array([p[1] for p in pnts], dtype=np.float64)

        tot_mass = np.trapz(y, x=x)
        idx = 0
        perc = 0.0
        while perc < 99.99999999:
            mass = np.trapz(y[:idx], x=x[:idx])
            idx += 1
            perc = (mass / tot_mass) * 100

        x_data = x[:idx]
        y_data = y[:idx]

        type(x_data)

        k0 = np.array([0.0, 0.0, 0.0], dtype=np.float64)

        def fun(k): return sse(k, x_data, y_data)

        popt = minimize(fun, k0)
#         display(popt)
        xf = np.linspace(x_data.min(), x_data.max(), 100)
        yf = gamma_kernel(xf, *popt.x)

#         if (phi in phis[-2:] and h in vals[::2]):
#             plt.scatter(x_data, y_data, s=30)
#             plt.plot(xf, yf, '--', lw=1.2, label=r' $H = %d$km' % disp_func(h))
#             plt.legend()
#             # axs[i].set_ylabel('Mass/Area')
#             # axs[i].set_xlabel(r'Radius')
#             plt.title(r'$\phi \in %s$' % disp_func(h), fontsize=14)

        fit_df = fit_df.append(
            {'H': h, 'Phi': phi, 'a': popt.x[0], 'b': popt.x[1], 'c': popt.x[2], 'data': (x_data, y_data)}, ignore_index=True)
#     plt.tight_layout()
#     plt.savefig("../data/%s_trial_%d/weird_%d.pdf" %
#                 (param, trial, k), format='pdf')
#     plt.show()
display(fit_df)

,H,Phi,a,b,c,data
0,10000,"[-4.5,-4.07143)",-0.605038,-0.002126,188135.959386,"([1135.6134905855954, 1441.3944637051995, 1692..."
1,11500,"[-4.5,-4.07143)",-0.664083,-0.001480,160891.522993,"([1135.6134905855954, 1441.3944637051995, 1692..."
2,13000,"[-4.5,-4.07143)",0.316699,-0.001920,286.883813,"([1135.6134905855954, 1441.3944637051995, 1692..."
3,14500,"[-4.5,-4.07143)",-0.537994,-0.001110,48329.347770,"([1135.6134905855954, 1441.3944637051995, 1692..."
4,16000,"[-4.5,-4.07143)",-0.405035,-0.001045,17759.957388,"([1135.6134905855954, 1441.3944637051995, 1692..."
5,17500,"[-4.5,-4.07143)",-0.445743,-0.000928,20540.337549,"([1135.6134905855954, 1441.3944637051995, 1692..."
6,19000,"[-4.5,-4.07143)",-0.429032,-0.000857,16602.689080,"([1135.6134905855954, 1441.3944637051995, 1692..."
7,20500,"[-4.5,-4.07143)",-0.485831,-0.000766,21842.094733,"([1135.6134905855954, 1441.3944637051995, 1692..."
8,22000,"[-4.5,-4.07143)",-0.506193,-0.000702,22886.743455,"([1135.6134905855954, 1441.3944637051995, 1692..."
9,23500,"[-4.5,-4.07143)",-0.492969,-0.000663,19430.487681,"([1135.6134905855954, 1441.3944637051995, 1692..."


In [134]:
a_piv = fit_df.pivot_table(index='Phi', columns='H', values='a')
b_piv = fit_df.pivot_table(index='Phi', columns='H', values='b')
c_piv = fit_df.pivot_table(index='Phi', columns='H', values='c')
a_piv = a_piv.reindex(phis)
b_piv = b_piv.reindex(phis)
c_piv = c_piv.reindex(phis)

In [140]:
fig, axs = plt.subplots(3, 1, figsize=(
    8, 15), facecolor='w', edgecolor='k')
axs = axs.ravel()
for i, piv, p in zip([0, 1, 2], [a_piv, b_piv, c_piv], ['a', 'b', 'c']):
    c = axs[i].pcolor(piv.values, cmap='coolwarm')
    # cont = axs[i].contour(scipy.ndimage.zoom(piv.values, 3), colors='w')
    axs[i].set_yticks(np.arange(0.5, len(piv.index), 1))
    axs[i].set_yticklabels(piv.index)
    axs[i].set_xticks(np.arange(0.5, len(piv.columns), 1))
    axs[i].set_xticklabels(piv.columns, rotation=60)
    fig.colorbar(c, ax=axs[i])
    axs[i].grid(False)
    axs[i].set_title(p)
    # axs[i].clabel(cont, inline=1, fontsize=10)
    axs[i].set_xlabel("Column height (m)")
    axs[i].set_ylabel("Phi class")
# plt.locator_params(nbins=7)
plt.tight_layout()
plt.show()
plt.savefig("../data/%s_trial_%d/param_maps_cont_weights.png" %
            (param, trial), format='png')

FigureCanvasNbAgg()

c:\users\nmostert\envs\spew\lib\site-packages\matplotlib\font_manager.py:1328: UserWarning: findfont: Font family ['serif'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))


In [48]:
cf = ["%.2f" % c for c in centroids]
cf

['-4.29',
 '-3.86',
 '-3.43',
 '-3.00',
 '-2.57',
 '-2.14',
 '-1.71',
 '-1.29',
 '-0.86',
 '-0.43',
 '0.00',
 '0.43',
 '0.86',
 '1.29',
 '1.71',
 '2.14',
 '2.57',
 '3.00',
 '3.43',
 '3.86',
 '4.29']

In [64]:
def pivot_map(piv):
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    X = np.tile(piv.columns.values, (len(piv.columns.values),1))
    Y = np.tile(np.arange(len(piv.index)), (len(piv.index),1)).T
    Z = piv.values
    ax.plot_surface(X, Y, Z, cmap='coolwarm')
    ax.set_xlabel("Column Height (km)")
    ax.set_yticks(np.arange(0.5, len(piv.index), 1))
    ax.set_yticklabels(cf, rotation=-40)
    ax.set_ylabel(r"$\phi$")
    ax.view_init(azim=-25)

In [65]:
pivot_map(a_piv)
plt.title("a")
plt.savefig("../data/%s_trial_%d/param_maps_a_surf.png" %
            (param, trial), format='png')

c:\users\nmostert\envs\spew\lib\site-packages\matplotlib\pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

c:\users\nmostert\envs\spew\lib\site-packages\matplotlib\font_manager.py:1328: UserWarning: findfont: Font family ['serif'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))


In [83]:
pivot_map(-b_piv)
plt.title("-b")
plt.savefig("../data/%s_trial_%d/param_maps_b_surf.png" %
            (param, trial), format='png')

FigureCanvasNbAgg()

c:\users\nmostert\envs\spew\lib\site-packages\matplotlib\font_manager.py:1328: UserWarning: findfont: Font family ['serif'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))


In [67]:
pivot_map(c_piv)
plt.title("c")
plt.savefig("../data/%s_trial_%d/param_maps_c_surf.png" %
            (param, trial), format='png')

c:\users\nmostert\envs\spew\lib\site-packages\matplotlib\pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

c:\users\nmostert\envs\spew\lib\site-packages\matplotlib\font_manager.py:1328: UserWarning: findfont: Font family ['serif'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))


In [132]:
plt.close("all")

In [119]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4), facecolor='w', edgecolor='k', sharey=True)
for i, (k, p) in enumerate(a_piv.T.iterrows()): 
    ax1.plot(p.index, p.values, color=plt.cm.coolwarm(i**2))
ax1.set_title("Grain size vs parameter $a$")
ax1.set_xticks(np.arange(0.5, len(piv.index), 1))
ax1.set_xticklabels(cf, rotation=-50)
ax1.set_ylabel("$a$")
ax1.set_xlabel("$\phi$ centroid")
for i, (k, p) in enumerate(a_piv.iterrows()): 
    c = ax2.plot(p.index, p.values, color=plt.cm.coolwarm(i**2))
ax2.set_title("Column Height vs parameter $a$")
ax2.set_xlabel("Column Height (m)")
plt.tight_layout()
plt.savefig("../data/%s_trial_%d/param_maps_a_lines.png" %
            (param, trial), format='png')

FigureCanvasNbAgg()

c:\users\nmostert\envs\spew\lib\site-packages\matplotlib\font_manager.py:1328: UserWarning: findfont: Font family ['serif'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))


In [118]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4), facecolor='w', edgecolor='k', sharey=True)
for i, (k, p) in enumerate(b_piv.T.iterrows()): 
    ax1.plot(p.index, -p.values, color=plt.cm.coolwarm(i**2))
ax1.set_title("Grain size vs parameter $-b$")
ax1.set_xticks(np.arange(0.5, len(piv.index), 1))
ax1.set_xticklabels(cf, rotation=-50)
ax1.set_ylabel("$-b$")
ax1.set_xlabel("$\phi$ centroid")
for i, (k, p) in enumerate(b_piv.iterrows()): 
    c = ax2.plot(p.index, -p.values, color=plt.cm.coolwarm(i**2))
ax2.set_title("Column Height vs parameter $-b$")
ax2.set_xlabel("Column Height (m)")
plt.tight_layout()
plt.savefig("../data/%s_trial_%d/param_maps_b_lines.png" %
            (param, trial), format='png')

FigureCanvasNbAgg()

c:\users\nmostert\envs\spew\lib\site-packages\matplotlib\font_manager.py:1328: UserWarning: findfont: Font family ['serif'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))


In [117]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4), facecolor='w', edgecolor='k', sharey=True)
for i, (k, p) in enumerate(c_piv.T.iterrows()): 
    ax1.plot(p.index, p.values, color=plt.cm.coolwarm(i**2))
ax1.set_title("Grain size vs parameter $c$")
ax1.set_xticks(np.arange(0.5, len(piv.index), 1))
ax1.set_xticklabels(cf, rotation=-50)
ax1.set_ylabel("$c$")
ax1.set_xlabel("$\phi$ centroid")
for i, (k, p) in enumerate(c_piv.iterrows()): 
    c = ax2.plot(p.index, p.values, color=plt.cm.coolwarm(i**2))
ax2.set_title("Column Height vs parameter $c$")
ax2.set_xlabel("Column Height (m)")
plt.tight_layout()
plt.savefig("../data/%s_trial_%d/param_maps_c_lines.png" %
            (param, trial), format='png')

FigureCanvasNbAgg()

c:\users\nmostert\envs\spew\lib\site-packages\matplotlib\font_manager.py:1328: UserWarning: findfont: Font family ['serif'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))


In [136]:
entry = fit_df[(fit_df.Phi == phis[-1])]
fig, axs = plt.subplots(7, 3, figsize=(
    8, 12), facecolor='w', edgecolor='k', sharex=True, sharey=True)
axs = axs.ravel()
for i, (k, en) in enumerate(entry.iterrows()):
    a = en.a
    b = en.b
    c = en.c
    x = en['data'][0]
    y = en['data'][1]
    xf = np.linspace(x.min(), x.max(), 100)
    yf = gamma_kernel(xf, a, b, c)
    axs[i].plot(xf, yf, 'k', lw=1)
    axs[i].scatter(x, y, s=5, facecolors='none', edgecolors='r', label=r'$H$ = %s km' % str(disp_func(en.H)))
    axs[i].legend()
plt.tight_layout()
plt.savefig("../data/%s_trial_%d/anomaly_phi_last.pdf" %
            (param, trial), format='pdf')

FigureCanvasNbAgg()

c:\users\nmostert\envs\spew\lib\site-packages\matplotlib\font_manager.py:1328: UserWarning: findfont: Font family ['serif'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))


In [137]:
entry = fit_df[(fit_df.Phi == phis[-2])]
fig, axs = plt.subplots(7, 3, figsize=(
    8, 12), facecolor='w', edgecolor='k', sharex=True, sharey=True)
axs = axs.ravel()
for i, (k, en) in enumerate(entry.iterrows()):
    a = en.a
    b = en.b
    c = en.c
    x = en['data'][0]
    y = en['data'][1]
    xf = np.linspace(x.min(), x.max(), 100)
    yf = gamma_kernel(xf, a, b, c)
    axs[i].plot(xf, yf, 'k', lw=1)
    axs[i].scatter(x, y, s=5, facecolors='none', edgecolors='r', label=r'$H$ = %s km' % str(disp_func(en.H)))
    axs[i].legend()
plt.tight_layout()
plt.savefig("../data/%s_trial_%d/anomaly_phi_penum.pdf" %
            (param, trial), format='pdf')

FigureCanvasNbAgg()

c:\users\nmostert\envs\spew\lib\site-packages\matplotlib\font_manager.py:1328: UserWarning: findfont: Font family ['serif'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))


In [138]:
entry = fit_df[(fit_df.Phi == phis[0])]
fig, axs = plt.subplots(7, 3, figsize=(
    8, 12), facecolor='w', edgecolor='k', sharex=True, sharey=True)
axs = axs.ravel()
for i, (k, en) in enumerate(entry.iterrows()):
    a = en.a
    b = en.b
    c = en.c
    x = en['data'][0]
    y = en['data'][1]
    xf = np.linspace(x.min(), x.max(), 100)
    yf = gamma_kernel(xf, a, b, c)
    axs[i].plot(xf, yf, 'k', lw=1)
    axs[i].scatter(x, y, s=5, facecolors='none', edgecolors='r', label=r'$H$ = %s km' % str(disp_func(en.H)))
    axs[i].legend()
plt.tight_layout()

FigureCanvasNbAgg()

c:\users\nmostert\envs\spew\lib\site-packages\matplotlib\font_manager.py:1328: UserWarning: findfont: Font family ['serif'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))


In [139]:
entry = fit_df[(fit_df.H == 10000)]
fig, axs = plt.subplots(7, 3, figsize=(
    8, 12), facecolor='w', edgecolor='k', sharex=True, sharey=True)
axs = axs.ravel()
for i, (k, en) in enumerate(entry.iterrows()):
    a = en.a
    b = en.b
    c = en.c
    x = en['data'][0]
    y = en['data'][1]
    xf = np.linspace(x.min(), x.max(), 100)
    yf = gamma_kernel(xf, a, b, c)
    axs[i].plot(xf, yf, 'k', lw=1)
    axs[i].scatter(x, y, s=5, facecolors='none', edgecolors='r', label=r'$\phi \in %s$' % en.Phi)
    axs[i].legend()
plt.tight_layout()
plt.savefig("../data/%s_trial_%d/anomaly_H_outlier.pdf" %
            (param, trial), format='pdf')

FigureCanvasNbAgg()

c:\users\nmostert\envs\spew\lib\site-packages\matplotlib\font_manager.py:1328: UserWarning: findfont: Font family ['serif'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))


In [193]:
piv = -b_piv

In [304]:
def lognorm(x, mu, sig):
    return np.exp(((-np.log(x) - mu)**2)/(2*sig**2))

def logit(x, a, b):
    return ((b/a)*(x/a)**(b-1))/((1+((x/a)**b)**2))

def sigmoid(x, A, B):
    return A/(1+np.exp(B*x))

def lnsse(k, x_data, y_data):
    return sum((y_data - (lognorm(x_data, *k))) ** 2)

In [315]:
fig, axs = plt.subplots(7, 3, figsize=(
    8, 12), facecolor='w', edgecolor='k', sharex=True, sharey=True)
axs = axs.ravel()
for i, (k, row) in enumerate(piv.T.iterrows()):
    bins = np.linspace(0,1,len(centroids))
    x_vals = row.values/row.values.max()
    def lnfun(k): return lnsse(k, bins, x_vals)
    k0 = np.array([-10,0.2], dtype=np.float64)
    popt = minimize(lnfun, k0)
    display(popt)
    xf = np.linspace(0,1, 100)
    yf = lognorm(xf, *popt.x)
    axs[i].plot(bins, x_vals, 'r')
    axs[i].plot(xf, yf, 'b')
#     axs[i].set_xticklabels([])
plt.tight_layout()

c:\users\nmostert\envs\spew\lib\site-packages\matplotlib\pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

c:\users\nmostert\envs\spew\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  
c:\users\nmostert\envs\spew\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
c:\users\nmostert\envs\spew\lib\site-packages\scipy\optimize\optimize.py:663: RuntimeWarning: invalid value encountered in double_scalars
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]
c:\users\nmostert\envs\spew\lib\site-packages\numpy\core\_methods.py:26: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims)


      fun: inf
 hess_inv: array([[1, 0],
       [0, 1]])
      jac: array([nan, nan])
  message: 'Optimization terminated successfully.'
     nfev: 4
      nit: 0
     njev: 1
   status: 0
  success: True
        x: array([-10. ,   0.2])

      fun: inf
 hess_inv: array([[1, 0],
       [0, 1]])
      jac: array([nan, nan])
  message: 'Optimization terminated successfully.'
     nfev: 4
      nit: 0
     njev: 1
   status: 0
  success: True
        x: array([-10. ,   0.2])

...

      fun: inf
 hess_inv: array([[1, 0],
       [0, 1]])
      jac: array([nan, nan])
  message: 'Optimization terminated successfully.'
     nfev: 4
      nit: 0
     njev: 1
   status: 0
  success: True
        x: array([-10. ,   0.2])

      fun: inf
 hess_inv: array([[1, 0],
       [0, 1]])
      jac: array([nan, nan])
  message: 'Optimization terminated successfully.'
     nfev: 4
      nit: 0
     njev: 1
   status: 0
  success: True
        x: array([-10. ,   0.2])

      fun: inf
 hess_inv: array([[1, 0],
       [0, 1]])
      jac: array([nan, nan])
  message: 'Optimization terminated successfully.'
     nfev: 4
      nit: 0
     njev: 1
   status: 0
  success: True
        x: array([-10. ,   0.2])

      fun: inf
 hess_inv: array([[1, 0],
       [0, 1]])
      jac: array([nan, nan])
  message: 'Optimization terminated successfully.'
     nfev: 4
      nit: 0
     njev: 1
   status: 0
  success: True
        x: array([-10. ,   0.2])

c:\users\nmostert\envs\spew\lib\site-packages\matplotlib\font_manager.py:1328: UserWarning: findfont: Font family ['serif'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))


In [111]:
plt.close("all")

In [266]:
fig, axs = plt.subplots(7, 3, figsize=(
    8, 12), facecolor='w', edgecolor='k', sharex=False, sharey=False)
axs = axs.ravel()
for i, (k, row) in enumerate(piv.T.iterrows()):
    bins = np.linspace(0,1,len(centroids))
    restored = [np.abs(c*v)+0.001 for c, v in zip(bins, row.values)]
    fit = stats.logistic.fit(restored, floc=0)
    print(fit)
    dist = stats.logistic(*stats.logistic.fit(restored, floc=0))
    x = bins
    y = dist.pdf(x)
    axs[i].plot(x,y,'b',linewidth=2)
    axs[i].plot(bins,row.values,'r',linewidth=2)
plt.show()

FigureCanvasNbAgg()

(0, 0.0008023725338173836)
(0, 0.000777491810171336)
(0, 0.0007587430606621645)
(0, 0.0007439444398098753)
(0, 0.0007350620091296466)
(0, 0.0007245712546956714)
(0, 0.0007198924594941736)
(0, 0.0007145124756940395)
(0, 0.0007087654783824154)
(0, 0.0007055621874387898)
(0, 0.0007031879147401223)
(0, 0.0007010500658831251)
(0, 0.0006968092656446778)
(0, 0.0006951006267295219)
(0, 0.000691486771420435)
(0, 0.0006896949528318107)
(0, 0.0006880268285038701)
(0, 0.000685757007840068)
(0, 0.0006874028645025569)
(0, 0.0006853325449860314)
(0, 0.0006828185850633944)


c:\users\nmostert\envs\spew\lib\site-packages\matplotlib\font_manager.py:1328: UserWarning: findfont: Font family ['serif'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))
